In [138]:
import os
import csv
import numpy as np
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Data Processing

In [189]:
directory = './col_lab/'
col_type = []
with open(directory+'labels_cur.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        col_type.append((row[0], row[1], row[2], row[3], row[4]))

In [260]:
trainable = []
all_trainable = []
for val in col_type:
    tp = val[2]
    all_trainable.append((val[0], val[4], val[2], val[3]))
    if tp == 'business_name' or tp == 'school_name' or tp == 'park_playground':
        trainable.append((val[0], val[4], val[2], val[3]))

In [269]:
os.path.exists('word_vec.model') == False

True

In [272]:
directory = './col_lab/labels'
all_train_files = {}
all_name_count = {i[2]:{'count':0, 'files': []} for i in all_trainable}
for val in all_trainable:
    file_id = val[1]
    if len(file_id) == 1:
        file_id = '00'+file_id
    elif len(file_id) == 2:
        file_id = '0'+file_id
    filename = val[0]+'_'+file_id
    file_name = directory+'/'+filename+'.csv'
    if os.path.exists(file_name) == False:
        file_name = './col_lab/labels_str_new/'+filename+'.csv'
    if os.path.exists(file_name):
        with open(file_name) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            next(csv_reader)
            file = {'name': filename, 'count':0, 'val':[], 'label':[]}
            for row in csv_reader:
                vals = row[0].strip().replace('-', ' ').split(' ')
                file['val'].append(vals)
                file['label'].append(row[1])
            file['count'] = len(file['val'])
            all_name_count[val[2]]['files'].append((filename, file['count']))
            all_name_count[val[2]]['count'] += file['count']
            all_train_files[filename] = file

In [191]:
directory = './col_lab/labels'
train_files = {}
name_count = {'business_name':{'count':0, 'files': []}, 'school_name':{'count':0, 'files': []}, 'park_playground':{'count':0, 'files': []}}
for val in trainable:
    file_id = val[1]
    if len(file_id) == 1:
        file_id = '00'+file_id
    elif len(file_id) == 2:
        file_id = '0'+file_id
    filename = val[0]+'_'+file_id
    with open(directory+'/'+filename+'.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        next(csv_reader)
        file = {'name': filename, 'count':0, 'val':[], 'label':[]}
        for row in csv_reader:
            vals = row[0].strip().replace('-', ' ').split(' ')
            if len(vals) >=3:
                file['val'].append(vals)
                file['label'].append(row[1])
        file['count'] = len(file['val'])
        name_count[val[2]]['files'].append((filename, file['count']))
        name_count[val[2]]['count'] += file['count']
        train_files[filename] = file

In [84]:
name_count

{'business_name': {'count': 17130,
  'files': [('2bmr-jdsv_000', 157),
   ('2v9c-2k7f_003', 110),
   ('43nn-pn8j_008', 11124),
   ('ci93-uc8s_046', 623),
   ('qcdj-rwhu_097', 345),
   ('tg3t-nh4h_106', 2888),
   ('w9ak-ipjd_153', 1883)]},
 'school_name': {'count': 21332,
  'files': [('2xh6-psuq_004', 1683),
   ('6je4-4x7e_018', 1603),
   ('cznr-hmrv_051', 1382),
   ('diks-hcwd_053', 1454),
   ('pchn-eaxn_090', 2929),
   ('pdpg-nn8i_091', 1452),
   ('sybh-s59s_105', 1460),
   ('tmr6-dfvn_107', 2934),
   ('upwt-zvh3_113', 1509),
   ('wks3-66bn_128', 1388),
   ('sqcr-6mww_190', 950),
   ('hy4q-igkk_194', 1389),
   ('sxmw-f24h_195', 1199)]},
 'park_playground': {'count': 9957,
  'files': [('sxx4-xhzg_103', 192),
   ('sxx4-xhzg_103', 192),
   ('sxmw-f24h_195', 1199),
   ('uzcy-9puk_202', 2149),
   ('uzcy-9puk_202', 2149),
   ('aiww-p3af_203', 2038),
   ('aiww-p3af_203', 2038)]}}

In [ ]:
train_names = ['tg3t-nh4h_106', 'pchn-eaxn_090', 'uzcy-9puk_202']

# Train Word2Vec and Doc2Vec and Prepare features

In [124]:
files = [i[0] for j in name_count for i in name_count[j]['files']]
sentences = []
for file in files:
    sentences += train_files[file]['val']

In [125]:
len(sentences)

48419

In [131]:
wv_model = Word2Vec(sentences, sg=1, size=100, window=5, min_count=1)
wv_model.save("word2vec.model")

In [132]:
wv_model.wv['LLC']

array([ 0.08794444,  0.22134978, -0.0092202 ,  0.24054499,  0.2705034 ,
       -0.0673551 , -0.30584073, -0.471543  ,  0.35528842, -0.3693265 ,
       -0.50038296,  0.21671194, -0.42957076, -0.09745683,  0.14815505,
        0.13660587,  0.07366879, -0.04761224,  0.14174868,  0.4221395 ,
        0.15434976,  0.19260518, -0.6081785 , -0.41330507, -0.08333189,
        0.09689377,  0.3075576 ,  0.2537698 , -0.22530046, -0.16096999,
       -0.3317518 , -0.20592462, -0.46064785,  0.48434308, -0.37788475,
        0.1692698 , -0.0295064 ,  0.06182968,  0.24920659,  0.00878918,
        0.24934027,  0.00285243, -0.08500164,  0.02877139,  0.02364055,
       -0.11218129,  0.01444972, -0.24496007,  0.15213022, -0.10469245,
       -0.0441895 , -0.12661783,  0.07576437,  0.07547969,  0.18130472,
        0.16276176,  0.20064329, -0.25726363,  0.02903674,  0.26573703,
       -0.15379214, -0.0283741 , -0.20315857, -0.1791972 , -0.03804828,
       -0.48056817, -0.3216531 , -0.28046852,  0.15400589,  0.12

In [128]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]

In [181]:
dv_model = Doc2Vec(documents, vector_size=100, dbow_words=1, window=2, min_count=1)
dv_model.save("doc2vec.model")

In [183]:
dv_model.infer_vector(['Chinese', 'Restaurants'])
# try:
#     wv_model.wv['1111111111']
# except:
#     print(1)

array([ 0.02432786,  0.00671296,  0.01756264,  0.00315118, -0.00920516,
       -0.0046181 , -0.01185251, -0.00885352,  0.01463391, -0.01515639,
       -0.0156415 ,  0.01903179, -0.00299502, -0.00206756,  0.01076518,
        0.00421982,  0.00037046,  0.00551941,  0.00562842,  0.01746982,
        0.00861378, -0.00765877, -0.03054351, -0.01446205, -0.0056932 ,
        0.02271719,  0.02139705,  0.00877519, -0.01864341,  0.0212835 ,
        0.002142  , -0.03463773,  0.01375672,  0.00631976, -0.00902638,
       -0.00669712,  0.02104242,  0.02200629, -0.01583948,  0.01031362,
       -0.00611978, -0.01115874,  0.00275569, -0.01603691,  0.00903394,
       -0.01203439, -0.02247518,  0.01777174,  0.02019343,  0.00753158,
       -0.0030833 ,  0.00123608, -0.01432449, -0.02428816, -0.0009323 ,
        0.01942965,  0.0106469 ,  0.00055742, -0.00638309,  0.01983718,
       -0.00490953, -0.01879386,  0.02069506, -0.00517643,  0.00409388,
       -0.00145344, -0.01980117,  0.00301836, -0.00889048,  0.01

In [196]:
feature_label = train_files
count = 0
for key in feature_label:
    feature_label[key]['features'] = []
    feature_label[key]['d2v'] = []
    w2vs = None
    for val in feature_label[key]['val']:
        count += 1
        if count % 5000 == 0:
            print("Processed items:", count)
        for word in val:
            if w2vs is None:
                w2vs = wv_model.wv[word]
            else:
                w2vs = np.vstack((w2vs, wv_model.wv[word]))
        w2v = np.mean(w2vs, axis=0)
        d2v = dv_model.infer_vector(val)
        feature  = list(w2v)+list(d2v)
        feature_label[key]['d2v'].append(d2v)
        feature_label[key]['features'].append(feature)

Processed items: 5000
Processed items: 10000
Processed items: 15000
Processed items: 20000
Processed items: 25000
Processed items: 30000
Processed items: 35000
Processed items: 40000


# Prepare train data

In [198]:
train_names = ['tg3t-nh4h_106', 'pchn-eaxn_090', 'uzcy-9puk_202']
test_names = [i for i in files if i not in train_names]
print(len(train_names), len(test_names), len(files))

3 23 27


In [217]:
X_train = []
y_train = []
for name in train_names:
    X_train += train_files[name]['features']
    y_train += train_files[name]['label']
print(len(X_train), len(y_train))
X_train = np.array(X_train)
y_train = np.array(y_train)
y_train = y_train.reshape(y_train.shape[0],1)
print(X_train.shape, y_train.shape)

7966 7966
(7966, 200) (7966, 1)


# Get Model

In [219]:
from sklearn.ensemble import RandomForestClassifier

In [221]:
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [255]:
dict_true_pred_labels = {}
for name in test_names:
    X_test = np.array(train_files[name]['features'])
    y_test = np.array(train_files[name]['label'])
    y_test = y_test.reshape(y_test.shape[0],1)
#     print(X_test.shape, y_test.shape)
    y_pred = clf.predict(X_test)
#     print(y_pred.shape)
    uniq_true, count_true = np.unique(y_test, return_counts=True)
    uniq_pred, count_pred = np.unique(y_pred, return_counts=True)
    dict_true_pred_labels[name] = {}
    dict_true_pred_labels[name]['true'] = {i:j for i in uniq_true for j in count_true}
    dict_true_pred_labels[name]['pred'] = {i:j for i,j in zip(uniq_pred, count_pred)}

In [256]:
dict_true_pred_labels

{'2bmr-jdsv_000': {'true': {'business_name': 157},
  'pred': {'school_name': 157}},
 '2v9c-2k7f_003': {'true': {'business_name': 110},
  'pred': {'school_name': 110}},
 '43nn-pn8j_008': {'true': {'business_name': 11124},
  'pred': {'business_name': 9935, 'school_name': 1189}},
 'ci93-uc8s_046': {'true': {'business_name': 623},
  'pred': {'business_name': 623}},
 'qcdj-rwhu_097': {'true': {'business_name': 345},
  'pred': {'business_name': 280, 'school_name': 65}},
 'w9ak-ipjd_153': {'true': {'business_name': 1883},
  'pred': {'business_name': 1844, 'school_name': 39}},
 '2xh6-psuq_004': {'true': {'school_name': 1683},
  'pred': {'business_name': 13, 'school_name': 1670}},
 '6je4-4x7e_018': {'true': {'school_name': 1603},
  'pred': {'park_playground': 1600, 'school_name': 3}},
 'cznr-hmrv_051': {'true': {'school_name': 1382},
  'pred': {'park_playground': 1382}},
 'diks-hcwd_053': {'true': {'school_name': 1454},
  'pred': {'park_playground': 1454}},
 'pdpg-nn8i_091': {'true': {'school_n

In [276]:
{i[2]:{'count':0, 'files': []} for i in all_trainable}

{'business_name': {'count': 0, 'files': []},
 'person_name': {'count': 0, 'files': []},
 'school_name': {'count': 0, 'files': []},
 'area_of_study': {'count': 0, 'files': []},
 'borough': {'count': 0, 'files': []},
 'building_classification': {'count': 0, 'files': []},
 'city_agency': {'count': 0, 'files': []},
 'school_level': {'count': 0, 'files': []},
 'website': {'count': 0, 'files': []},
 'neighborhood': {'count': 0, 'files': []},
 'subject_in_school': {'count': 0, 'files': []},
 'phone_number': {'count': 0, 'files': []},
 'lat_lon_cord': {'count': 0, 'files': []},
 'address': {'count': 0, 'files': []},
 'street_name': {'count': 0, 'files': []},
 'Bulding classification': {'count': 0, 'files': []},
 'color': {'count': 0, 'files': []},
 'car_make': {'count': 0, 'files': []},
 'park_playground': {'count': 0, 'files': []},
 'zip_code': {'count': 0, 'files': []},
 'location_type': {'count': 0, 'files': []},
 'vehicle_type': {'count': 0, 'files': []},
 'address_number': {'count': 0, 'f

In [287]:
all_name_count['car_make']

{'count': 42058,
 'files': [('n84m-kx4j_086', 35),
  ('vr8p-8shw_121', 1854),
  ('pq5i-thsu_156', 6637),
  ('bjuu-44hx_172', 6318),
  ('pvqr-7yc4_188', 1630),
  ('jt7v-77mi_211', 6870),
  ('kiv2-tbus_217', 6030),
  ('2bnn-yakx_219', 5701),
  ('c284-tqph_222', 6983)]}

In [323]:
directory = './col_lab/labels'
car_make = []
for val in ['n84m-kx4j_086', 'vr8p-8shw_121', 'pq5i-thsu_156', 'bjuu-44hx_172', 'pvqr-7yc4_188', \
            'jt7v-77mi_211', 'kiv2-tbus_217', '2bnn-yakx_219', 'c284-tqph_222']:
    file_name = directory+'/'+val+'.csv'
    if os.path.exists(file_name) == False:
        file_name = './col_lab/labels_str_new/'+filename
    if os.path.exists(file_name):
        with open(file_name) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            next(csv_reader)
            for row in csv_reader:
                car_make.append(row[0].strip().lower())

In [324]:
set_car_make = set(car_make)

In [298]:
import pandas as pd

In [334]:
tsv_table=pd.read_csv('task2Data/as69-ew8f.tsv', sep='\t', header=0)
tsv_columns=tsv_table.columns
truck_make = tsv_table['TruckMake'].to_list()
semantic_type = {'car_make':0, 'car_model':0, 'license_plate':0}
other = []
for i in truck_make:
    i_split = i.lower().split(" ")
    if len(i_split) == 2 and i_split[0] in ['ram', 'ford']:
        semantic_type['car_model'] += 1
    elif i_split[0] == 'ag16439':
        semantic_type['license_plate'] += 1
    else:
        semantic_type['car_make'] += 1
        other.append(i)

In [335]:
semantic_type

{'car_make': 118816, 'car_model': 719, 'license_plate': 1}

In [340]:
tsv_table=pd.read_csv('task2Data/ph7v-u5f3.tsv', sep='\t', header=0)
tsv_columns=tsv_table.columns
truck_make = tsv_table['TOP VEHICLE MODELS - 5'].dropna().to_list()
semantic_type = {'car_make':0, 'car_model':0, 'license_plate':0}
other = []
for i in truck_make:
    if i.strip().lower() in set_car_make:
        semantic_type['car_make'] += 1
    else:
        semantic_type['car_model'] += 1
        other.append(i)
#     i_split = i.lower().split(" ")
#     if len(i_split) == 2 and i_split[0] in ['ram', 'ford']:
#         semantic_type['car_model'] += 1
#     elif i_split[0] == 'ag16439':
#         semantic_type['license_plate'] += 1
#     else:
#         semantic_type['car_make'] += 1
#         other.append(i)

In [345]:
semantic_type

{'car_make': 1, 'car_model': 646, 'license_plate': 0}